# Predicting Sentiment Levels in Yelp Reviews
## STAT 628 Mon, Group 6, Module 2

## Data Overview and Cleaning
### Uninformative reviews
There are some reviews that provide invalid information:

(Show examples)

These kind of reviews consist only a minority of the whole dataset (less than 0.1%), a simple solution is to delete them from our training set.

### Translation
More than 99% of the reviews were written in English while the rest of the reviews are in multiple languages including French, Japanese, German, Chinese (Simplified, Traditional and Cantonese). Although we could also delete them, they are mostly well written reviews and contain beneficial information for our future models. Therefore, we decided to translate all the reviews 

(EXAMPLES!)

The information that one review is written in a non-English language wasn't used as a feature label since doing a chi-square test, the results demonstrate that there isn't a significant difference in the distribution of stars between a English review and a non-English review.

(TEST RESULTS

## Model
### Redefine the Problem

Inspired by the trending Kaggle competition on predicting toxic comments (LINK!), we redefined the problem to a binary classification problem by asking the question "Is the review a 1/2/3/4/5 star review?" or "What is the probability that the review is a 1/2/3/4/5 star review?" five times instead of predicting the stars of the reviews in one run. This process is also know as Error- Correcting Output Coding (ECOC) introduced by  

There are many advantages under the new framework. For example, there are many methods available methods to do a binary classification problem and they usually run faster. Moreover, it provides a natural smoothing procedure by calculating the estimated star using the mean or expectation which is more likely to decrease the RMSE. 



### NB-SVM
NB-SVM, short for Naive Bayes Support Vector Machines

(Explain it

### Xgboost

## Ensemble
Empirically, ensembling results of different methods where the predictions are relatively accurate and uncorrelated will improve the prediction performance.

(Make examples misclassified by current classifier more important in the next classifier
(Create classifiers using different training sets, where each training set is created by “bootstrapping,” i.e., drawing examples (with replacement) from all possible training examples

## Conclusion

## Reference
1. https://github.com/sidaw/nbsvm
2. <a id='another_cell'></a> 